In [3]:
import numpy as np
import h5py
# import pynwb
import dendro.client as prc
import remfile
import kachery_cloud as kcl
import figurl


# Load project D-000128
project = prc.load_project('a016a71a')

# Lazy load imported/000128/sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb
nwb_file = project.get_file('imported/000128/sub-Jenkins/sub-Jenkins_ses-full_desc-train_behavior+ecephys.nwb')
nwb_remf = remfile.File(nwb_file)
nwb_h5 = h5py.File(nwb_remf, 'r')
# io = pynwb.NWBHDF5IO(file=nwb_h5, mode='r')
# nwb = io.read()

In [4]:
timestamps = nwb_h5['/processing/behavior/cursor_pos/timestamps'][:]
cursor_pos = nwb_h5['/processing/behavior/cursor_pos/data'][:]
spike_times = nwb_h5['/units/spike_times'][:]
spike_times_index = nwb_h5['/units/spike_times_index'][:]
unit_ids = nwb_h5['/units/id'][:]

In [7]:
place_fields = []
def _get_spike_times_indices(spike_times, timestamps):
    return np.searchsorted(timestamps, spike_times, side='left')
for j in range(len(unit_ids)):
    unit_id = unit_ids[j]
    i1 = 0 if j == 0 else spike_times_index[j-1]
    i2 = spike_times_index[j]
    spike_times0 = spike_times[i1:i2]
    spike_times0_indices = _get_spike_times_indices(spike_times0, timestamps)
    cursor_pos0 = cursor_pos[spike_times0_indices]
    place_fields.append({
        'unitId': unit_id,
        'x': cursor_pos0[:, 0].astype(np.float32),
        'y': cursor_pos0[:, 1].astype(np.float32)
    })
data = {
    'type': 'place_fields',
    'placeFields': place_fields
}
data_uri = kcl.store_json(figurl.serialize_data(data))
# fi-sci figurl-spike-sorting-analysis
view_url = f'https://figurl.org/f?v=https://figurl-place-fields.surge.sh&d={data_uri}'
print(view_url)

from IPython.display import IFrame
IFrame(src=view_url + '&hide=1', width='100%', height=600)

https://figurl.org/f?v=https://figurl-place-fields.surge.sh&d=sha1://388c94249c798d28a94d02eeefae7c868b21efcf
